In [1]:
from collections import namedtuple
import random

from gym.wrappers import RescaleAction, TimeLimit
import json
from matplotlib.patches import Ellipse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import Bounds, minimize
import seaborn as sns
from stable_baselines3 import PPO, TD3
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from tqdm import tqdm

from environments import ARESEAOptimization, ARESEASequential, ResetActuators, ResetActuatorsToDFD

In [2]:
def pack_dataframe(fn):
    def wrapper(env, problem=None):
        observations, incoming, misalignments = fn(env, problem=problem)
        observations = np.array(observations)

        df = pd.DataFrame(np.arange(len(observations)), columns=["step"])
        df["q1"] = observations[:,0]
        df["q2"] = observations[:,1]
        df["q3"] = observations[:,2]
        df["cv"] = observations[:,3]
        df["ch"] = observations[:,4]
        df["mup_x"] = observations[:,5]
        df["mup_y"] = observations[:,6]
        df["sigmap_x"] = observations[:,7]
        df["sigmap_y"] = observations[:,8]
        df["mu_x"] = observations[:,9]
        df["mu_y"] = observations[:,10]
        df["sigma_x"] = observations[:,11]
        df["sigma_y"] = observations[:,12]
        for k in incoming.keys():
            df["incoming_"+k] = float(incoming[k])
        df["misalignment_q1_x"] = misalignments[0]
        df["misalignment_q1_y"] = misalignments[1]
        df["misalignment_q2_x"] = misalignments[2]
        df["misalignment_q2_y"] = misalignments[3]
        df["misalignment_q3_x"] = misalignments[4]
        df["misalignment_q3_y"] = misalignments[5]
        df["misalignment_screen_x"] = misalignments[6]
        df["misalignment_screen_y"] = misalignments[7]

        return df
    
    return wrapper

In [3]:
@pack_dataframe
def run(env, problem=None):
    if problem is not None:
        if "initial" in problem:
            env.unwrapped.next_initial = problem["initial"]
        if "incoming" in problem:
            env.unwrapped.backend.next_incoming = problem["incoming"]
        if "misalignments" in problem:
            env.unwrapped.backend.next_misalignments = problem["misalignments"]
        if "desired" in problem:
            env.unwrapped.next_desired = problem["desired"]

    observations = []

    observation = env.reset()

    observations.append(observation)
    incoming = env.backend._incoming.parameters
    misalignments = env.backend.misalignments

    def optfn(actuators):
        observation, objective, _, _ = env.step(actuators)
        observations.append(observation)
        return objective

    # bounds = Bounds(env.action_space.low, env.action_space.high)
    # TODO: Should probably be options={"fatol": 4.5e-11}
    #       Because max pixel_size = 3.3198e-6
    #       Double that (two pixels) -> 6.6396e-06
    #       Squared error would then be 4.408428816e-11
    #       Round to 4.5e-11
    # minimize(optfn, observation[:5], method="Nelder-Mead", bounds=bounds, options={"fatol": 4.5e-11, "xatol": 1})

    return observations, incoming, misalignments

In [4]:
def cache_to_file(fn):
    def wrapper(method, **kwargs):
        filename = f".cache_3/{method}.pkl"
        
        try:
            evaluation = pd.read_pickle(filename)
            print(f"Read {method} from cache file")
        except FileNotFoundError:
            evaluation = fn(method, **kwargs)
            evaluation.to_pickle(filename)
        
        return evaluation

    return wrapper

In [7]:
@cache_to_file
def evaluate(method, description=None):
    env = ARESEAOptimization()

    with open("problems_3.json", "r") as f:
        problems = json.load(f)
    
    if "fdf" in method:
        env = ResetActuatorsToDFD(env)
    elif "zero" in method:
        env = ResetActuators(env)

    evaluation = []
    for i, problem in enumerate(tqdm(problems)):
        result = run(env, problem=problem)
        result["problem"] = i
        evaluation.append(result)
    evaluation = pd.concat(evaluation)
    evaluation["method"] = method
    evaluation["model"] = method
    if description is not None:
        evaluation["description"] = description
    
    return evaluation

In [8]:
evaluate("nothing", description="Do Nothing"),
evaluate("set-fdf-10", description="Set to FDF (k1=10)")
evaluate("set-zero", description="Set to Zero")

Read nothing from cache file
Read set-fdf-10 from cache file


100%|██████████| 300/300 [00:14<00:00, 20.52it/s]


,step,q1,q2,q3,cv,ch,mup_x,mup_y,sigmap_x,sigmap_y,...,misalignment_q2_x,misalignment_q2_y,misalignment_q3_x,misalignment_q3_y,misalignment_screen_x,misalignment_screen_y,problem,method,model,description
0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000299,0.000217,0.000237,0.000274,0.000374,-0.000046,0,set-zero,set-zero,Set to Zero
0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000500,0.000000,...,0.000070,0.000019,0.000161,-0.000335,0.000091,-0.000219,1,set-zero,set-zero,Set to Zero
0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000500,...,-0.000281,-0.000361,0.000123,0.000162,-0.000283,0.000049,2,set-zero,set-zero,Set to Zero
0,0,0.0,0.0,0.0,0.0,0.0,-0.001000,0.000000,0.000000,0.000000,...,0.000234,-0.000048,-0.000012,0.000040,-0.000298,-0.000016,3,set-zero,set-zero,Set to Zero
0,0,0.0,0.0,0.0,0.0,0.0,-0.001000,0.000000,0.000500,0.000000,...,-0.000123,0.000104,-0.000356,0.000006,0.000209,0.000141,4,set-zero,set-zero,Set to Zero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,0.0,0.0,0.0,0.0,0.0,-0.000409,0.000861,0.000058,0.000087,...,0.000223,0.000103,0.000191,-0.000330,0.000298,0.000387,295,set-zero,set-zero,Set to Zero
0,0,0.0,0.0,0.0,0.0,0.0,-0.000285,0.001050,0.000225,0.000293,...,0.000196,-0.000366,0.000168,-0.000379,0.000164,0.000008,296,set-zero,set-zero,Set to Zero
0,0,0.0,0.0,0.0,0.0,0.0,0.001082,-0.001381,0.000332,0.000036,...,0.000352,-0.000266,-0.000022,0.000398,0.000022,-0.000059,297,set-zero,set-zero,Set to Zero
0,0,0.0,0.0,0.0,0.0,0.0,0.000755,-0.000674,0.000323,0.000175,...,-0.000201,-0.000289,-0.000218,0.000151,0.000331,-0.000152,298,set-zero,set-zero,Set to Zero
